In [1]:
from selenium import webdriver
import csv
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [2]:
link_product = [];
with open('link_san_pham_tiki.csv', newline='', encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        link_product.append(row['link'])

In [3]:
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import xlwt
from xlwt import Workbook

def layBinhLuan(driver,star):
    # Thực hiện lấy bình luận tại các trang, bắt đầu từ trang 1
    p = 1
    list_comments = []
    while(True):
        flag = False
        #Xác định các trang bình luận
        time.sleep(1)
        pages = driver.find_elements(By.XPATH,"//div[@class='customer-reviews__pagination']//ul//a")
        for page in pages:
            # Duyệt lần lượt các trang, nếu đúng trang cần lấy thì thực hiện click chuột vào
            if(page.text.strip()==str(p)):
                time.sleep(1) #Tạm dừng 2 giây để website load dữ liệu
                page.click()
                flag = True
                break
        if(p == 1 ):
            flag = True
        if(flag == True):
            content = driver.page_source #Lấy lại nội dung mới của website sau khi thực hiện click vào các trang
            soup = BeautifulSoup(content,'html.parser') #Sử dụng BeautifulSoup để xử lý
            breadcrumb = soup.find('div',attrs={'data-view-id':'breadcrumb_container'}).findAll('a',attrs={'class':'breadcrumb-item'})
            if(breadcrumb):
                product_name = breadcrumb[-1].text
            else:
                product_name = ""
            
            # Xác định các thẻ div chứa bình luận 
            comments = soup.find('div',attrs={'class':'customer-reviews__inner'}).findAll('div',attrs={'class':'review-comment'})

            # Duyệt lần lượt từng thẻ div để lấy dữ liệu
            for com in comments:
                content_comment = com.find('div',attrs={'class':'review-comment__content'}).text.strip()
                list_comments.append([star,content_comment, product_name]) # Lưu dữ liệu vào mảng list_comments gồm số sao bình chọn và nội dung bình luận
            p = p + 1 # Tăng biến p để lấy trang tiếp theo
        else:
            break
    return list_comments


In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
list_comments = []
count = 0
for link in link_product:
    driver.get(link)

    # Điều khiển di chuyển đến vị trí các bình luận
    for i in range(8):
        driver.execute_script("window.scrollTo(0,"+str(i*710)+")") 
        time.sleep(0.5)
    try:
        element = WebDriverWait(driver, 7).until(
            EC.presence_of_element_located((By.CLASS_NAME, "filter-review__inner"))
        )
        
         # Tìm biểu tượng lọc 1 sao và điều khiển click chuột để chỉ hiển thị các bình luận 1 sao
        one_star = driver.find_element(By.XPATH,"//div[@class='filter-review__inner']//div[@data-view-index='7']")
        one_star.click();
        one_star_comment = layBinhLuan(driver,1) #Lấy bình luận 1 sao
        list_comments = list_comments + one_star_comment
        one_star.click(); #bỏ lọc 1 sao
        time.sleep(1)
        
        # Tìm biểu tượng lọc 2 sao và điều khiển click chuột để chỉ hiển thị các bình luận 2 sao
        two_star = driver.find_element(By.XPATH,"//div[@class='filter-review__inner']//div[@data-view-index='6']")
        two_star.click();
        two_star_comment = layBinhLuan(driver,2)#Lấy bình luận 2 sao
        list_comments = list_comments + two_star_comment
        two_star.click(); #bỏ lọc 2 sao
        time.sleep(1)
        
        # Tìm biểu tượng lọc 3 sao và điều khiển click chuột để chỉ hiển thị các bình luận 3 sao
        three_star = driver.find_element(By.XPATH,"//div[@class='filter-review__inner']//div[@data-view-index='5']")
        three_star.click();
        three_star_comment = layBinhLuan(driver,3) #Lấy bình luận 3 sao
        list_comments = list_comments + three_star_comment
        three_star.click(); #bỏ lọc 3 sao
        time.sleep(1)
        
        # Tìm biểu tượng lọc 4 sao và điều khiển click chuột để chỉ hiển thị các bình luận 4 sao
        four_star = driver.find_element(By.XPATH,"//div[@class='filter-review__inner']//div[@data-view-index='4']")
        four_star.click();
        four_star_comment = layBinhLuan(driver,4) #Lấy bình luận 4 sao
        list_comments = list_comments + four_star_comment
        four_star.click(); #bỏ lọc 4 sao
        time.sleep(1)
        
        count = count + 1
        print(count)
        
        #Mỗi khi lấy bình luận từ 10 link thì lưu lại một lần phần nội dung bình luận lấy được
        if(count%10==0):
            wb = Workbook()
            sheet = wb.add_sheet('Sheet 1')
            sheet.write(0, 0, 'rating')
            sheet.write(0, 1, 'comment')
            sheet.write(0, 2, 'product_name')
            for k, r in enumerate(list_comments):
                for j in range(len(r)):
                    sheet.write(k+1,j,r[j])

            wb.save('list_comments_tiki_'+str(count)+'.xls')
    except:
        print("Error")
        
wb = Workbook()
sheet = wb.add_sheet('Sheet 1')
sheet.write(0, 0, 'rating')
sheet.write(0, 1, 'comment')
sheet.write(0, 2, 'product_name')
for k, r in enumerate(list_comments):
    for j in range(len(r)):
        sheet.write(k+1,j,r[j])

wb.save('list_comments_tiki_full.xls')

1
2
3
4
5


In [ ]:
wb = Workbook()
sheet = wb.add_sheet('Sheet 1')
sheet.write(0, 0, 'rating')
sheet.write(0, 1, 'comment')
sheet.write(0, 2, 'product_name')
for k, r in enumerate(list_comments):
    for j in range(len(r)):
        sheet.write(k+1,j,r[j])

wb.save('list_comments_tiki_full.xls')